# ML training:  Exact vs Iterative & Scikit-Learn

Acknowledgement: This notebook was derived in part from A. Geron's materials: https://github.com/ageron/handson-ml2/blob/master/04_training_linear_models.ipynb

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

### Get the data
Here we're manufacturing it.

We're going to keep this simple:

$$ y(x) = 4 + 2x + \text{noise}$$

In [ ]:
# this is our array of x values
x = np.linspace(0, 10, 50)

# this y is the underlying linear function
y_underlying = 4 + 2*x

# generate 50 points of noise from a normal 
# distribution that has mean = 0 and std dev = 1.5
np.random.seed(42)
noise = np.random.normal(0,1.5,50)

# this y is the theoretical value + noise
y = y_underlying + noise

In [ ]:
# plot our theory curve
plt.plot(x,y_underlying,'k')

# plot our data generated from the theory curve + noise
plt.scatter(x,y,color='k',marker='o')

plt.show()

# Using machine learning

When we have data and we want to use it to make predictions, or when we want to learn the best parameters for modeling relationships between our dependent and independent variables, we will not have something like "y_underlying".  We will only have "y".

Our "y" here is intended to represent a real noisy set of values of our dependent variable.  On the basis of our noisy "y" and the values of our independent variable "x", we can use machine learning to determine an optimum set of values for modeling the data.  

For linear regression, that is equivalent to finding the $\Theta$ that optimizes how well the data is fit when we specify that 

$$y_{pred} = \bf{\theta^T x}$$ 

or in our case 

$$y_{pred} = \theta_0x^0 + \theta_1x^1$$ 

or as commonly written for linear equations, 

$$y_{pred} = mx+b$$  

Here, we're going to compare the $m$ and $b$ that we find from using machine learning to the $m=2$ and $b=4$ from our underlying linear equation that was used to generate the data.

## Exact solution

* To get the bias term, we make array elements whose first term is 1 (for the bias) and whose second term is x (for the x-value)
* We also reshape y

In [ ]:
x

In [ ]:
y

In [ ]:
X_b = np.c_[np.ones((50, 1)), x] 
y = y.reshape(-1,1)

In [ ]:
X_b

In [ ]:
y

**The analytical solution**

$\theta = (X^TX)^{-1}X^Ty$

This is a straight-forward and direct calculation of $\theta$.  There isn't any optimization here, there's no trying to minimize the cost function, it's an exact solution.  However, it does rely on inverting a matrix, which can potentially be time-consuming and be ill-defined for certain types of matrices.

In [ ]:
theta_best = np.linalg.inv(X_b.T.dot(X_b)).dot(X_b.T).dot(y)

In [ ]:
theta_best

The above does not give exactly $m=2$ and $b=4$, but it gives the best solution for $\theta$ on the basis of the information contained in our noisy set of data.

To see how it compares with the data, we'll make a plot of the line with a scatter plot of the data.

In [ ]:
# Make a two-element array at the min and max of the x range.
X_new = np.array([[0], [10]])
X_new_b = np.c_[np.ones((2, 1)), X_new]

In [ ]:
X_new_b

In [ ]:
y_predict = X_new_b.dot(theta_best)
y_predict

In [ ]:
plt.plot(X_new, y_predict, "r-")
plt.plot(x, y, "b.")
plt.show()

## Doing this with Stochastic Gradient Descent (SGD)

SGD is a method that uses iteration to converge towards optimum values for $\theta$.

We'll start by randomly getting 2 values to initialize $\bf{\theta}$ (for $\theta_0$ and $\theta_1$, or equivalently for $b$ and $m$, depending on how you write the coefficients in the linear equation).

In [ ]:
np.random.randn(2,1)

### Batch Gradient Descent

For batch gradient descent, one iteration of updating the parameters is carried out by training on the entire data set.

The coefficients for our linear equation are updated using gradient descent

$$ \theta = \theta - \eta \frac{\partial J}{\partial \theta} $$

where $\eta$ is the learning rate and $J$ is the cost function that we are trying to minimize.  Here we use a cost function appropriate to the mean squared error.

In [ ]:
X_b

In [ ]:
eta = 0.01  # learning rate
n_iterations = 1000
m = 50

theta = np.random.randn(2,1)  # random initialization

for iteration in range(n_iterations):
    gradients = 2/m * X_b.T.dot(X_b.dot(theta) - y)
    theta = theta - eta * gradients

In [ ]:
theta

The above gives good agreement with our expected $m = 2$ and $b = 4$.

Let's plot the evolving linear fit for different values of $\eta$.

In [ ]:
for i in [0.001, 0.01, 0.05]:
    eta = i  # learning rate
    n_iterations = 1000
    m = 50

    theta = np.random.randn(2,1)  # random initialization

    for iteration in range(n_iterations):
        gradients = 2/m * X_b.T.dot(X_b.dot(theta) - y)
        theta = theta - eta * gradients
        if iteration < 20:
            X_new = np.array([[0], [10]])
            X_new_b = np.c_[np.ones((2, 1)), X_new]  # add x0 = 1 to each instance
            y_predict = X_new_b.T.dot(theta)
            plt.plot(X_new, y_predict, "r-")
            plt.plot(x, y, "b.")
    plt.show()

### Stochastic Gradient Descent

For stochastic gradient descent, one iteration of updating the parameters is carried out by training on a single point in the data set, rather than updating with *all* of the data at once.  This is a faster process but more stochastic.

In the code below, we also use a learning schedule hyperparameter.  This decreases $\eta$ after every update, so that the coefficients are more likely to converge toward the minimum rather than randomly bouncing around the minimum or even shooting over to another minimum.  On the other hand, it means that this method may be slower to converge if the learning becomes too slow before it actually gets to the minimum.

In [ ]:
theta_intercept_sgd = []
theta_slope_sgd = []
m = len(X_b)
np.random.seed(42)

n_epochs = 100
t0, t1 = 5, 50  # learning schedule hyperparameters

def learning_schedule(t):
    return 0.2 * t0 / (t + t1)

theta = np.random.randn(2,1)  # random initialization

for epoch in range(n_epochs):
    for i in range(m):
        if epoch == 0 and i < 20:                    # not shown in the book
            y_predict = X_new_b.T.dot(theta)           # not shown
            style = "b-" if i > 1 else "r--"         # not shown
            plt.plot(X_new, y_predict, style)        # not shown
        random_index = np.random.randint(m)
        xi = X_b[random_index:random_index+1]
        yi = y[random_index:random_index+1]
        gradients = 2 * xi.T.dot(xi.dot(theta) - yi)
        eta = learning_schedule(epoch * m + i)
        theta = theta - eta * gradients
        theta_intercept_sgd.append(theta[0][0])
        theta_slope_sgd.append(theta[1][0])                 # not shown

plt.plot(x, y, "b.")                                 # not shown
plt.show()                                           # not shown

In [ ]:
theta

In [ ]:
len(theta_slope_sgd)

In [ ]:
fig,ax = plt.subplots(2,1)
ax[0].plot(theta_intercept_sgd)
ax[1].plot(theta_slope_sgd)

## The Scikit-Learn way for SGD

In [ ]:
from sklearn.linear_model import SGDRegressor

sgd_reg = SGDRegressor(max_iter=1000, tol=1e-3, penalty=None, eta0=0.1, random_state=42)
sgd_reg.fit(x.reshape(-1,1), y.ravel())

In [ ]:
sgd_reg.intercept_, sgd_reg.coef_

In [ ]:
plt.plot(x.reshape(-1,1), 
         y.ravel(),
         'b.')
plt.plot(np.array([[0], [10]]), 
         sgd_reg.predict(np.array([[0], [10]])), 
         'r-')

In [ ]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(x.reshape(-1,1), y.ravel())

In [ ]:
lin_reg.intercept_, lin_reg.coef_

In [ ]:
plt.plot(x.reshape(-1,1), 
         y.ravel(),
         'b.')
plt.plot(np.array([[0], [10]]), 
         lin_reg.predict(np.array([[0], [10]])), 
         'r-')
plt.plot(np.array([[0], [10]]), 
         sgd_reg.predict(np.array([[0], [10]])), 
         'g--')